# Receiving a P2SH(Multisig) - BX Exercise
In this exercise, we send a p2pkh input to a p2sh wrapped multisig output.

<hr style="border: 0.5px dashed #000;">

## 1. Restore wallet and generate spending key pairs.




In [18]:
# my_mnemonic ="word0 word1 word2 ..."
# hd_master_private=$( )
# hd_master_public=$( )

my_mnemonic="seven mail crash you unit small assume express wedding cloud work potato"
hd_master_private=$(bx mnemonic-to-seed $my_mnemonic | bx hd-new)
hd_master_public=$(bx hd-to-public $hd_master_private)


### Restore your spending keys.

We will be spending the funds controlled by these keys from account 1. 
* `m/44'/1'/1'/0/7`



In [19]:
# hd_m_44h_1h_1h_0_7=

# privatekey_44h_1h_1h_0_7=
# publickey_44h_1h_1h_0_7=
# publickeyhash_44h_1h_1h_0_7=
# address_44h_1h_1h_0_7=

hd_m_44h_1h_1h_0_7=$(bx hd-private --hard --index 44 $hd_master_private \
| bx hd-private --hard --index 1  \
| bx hd-private --hard --index 1  \
| bx hd-private --index 0 \
| bx hd-private --index 7)

privatekey_44h_1h_1h_0_7=$(bx hd-to-ec $hd_m_44h_1h_1h_0_7)
publickey_44h_1h_1h_0_7=$(bx hd-to-public $hd_m_44h_1h_1h_0_7 | bx hd-to-ec)
publickeyhash_44h_1h_1h_0_7=$(bx sha256 $publickey_44h_1h_1h_0_7 | bx ripemd160)
address_44h_1h_1h_0_7=$(bx hd-to-public $hd_m_44h_1h_1h_0_7 | bx hd-to-ec | bx ec-to-address --version 111)

echo $address_44h_1h_1h_0_7
echo $publickeyhash_44h_1h_1h_0_7

mjvWL2ECWonM4Dcs67rB1rFi3B5ro3tBzN
3054cf92dc80c6efd7559e660bde8b0fa7a85adc


### Generate three multisig key pairs from our 3rd wallet account.

We will be spending coin to a 2-of-3 output script controlled by these keys.

* `m/44'/1'/2'/0/0`
* `m/44'/1'/2'/0/1`
* `m/44'/1'/2'/0/2`



In [20]:
hd_m_44h_1h_2h_0_0=$(bx hd-private --hard --index 44 $hd_master_private \
| bx hd-private --hard --index 1  \
| bx hd-private --hard --index 2  \
| bx hd-private --index 0 \
| bx hd-private --index 0)
publickey_44h_1h_2h_0_0=$(bx hd-to-public $hd_m_44h_1h_2h_0_0 | bx hd-to-ec)

hd_m_44h_1h_2h_0_1=$(bx hd-private --hard --index 44 $hd_master_private \
| bx hd-private --hard --index 1  \
| bx hd-private --hard --index 2  \
| bx hd-private --index 0 \
| bx hd-private --index 1)
publickey_44h_1h_2h_0_1=$(bx hd-to-public $hd_m_44h_1h_2h_0_1 | bx hd-to-ec)

hd_m_44h_1h_2h_0_2=$(bx hd-private --hard --index 44 $hd_master_private \
| bx hd-private --hard --index 1  \
| bx hd-private --hard --index 2  \
| bx hd-private --index 0 \
| bx hd-private --index 3)
publickey_44h_1h_2h_0_2=$(bx hd-to-public $hd_m_44h_1h_2h_0_2 | bx hd-to-ec)

### Fetch previous UTXO's you are spending.

In [21]:
# bx fetch-tx [transaction_hash] -f json | jq ".transaction.outputs[output_index]"

bx fetch-tx 11a8587f5077fc7f69e3349797fd74f99864d27716d6b48ac51734cb3bec03ea -f json | jq ".transaction.outputs[7]"

{
  "address_hash": "3054cf92dc80c6efd7559e660bde8b0fa7a85adc",
  "script": "dup hash160 [3054cf92dc80c6efd7559e660bde8b0fa7a85adc] equalverify checksig",
  "value": "499550"
}


In [22]:
# previous_txid=
# previous_output_index=
# previous_output_amount=

previous_txid=11a8587f5077fc7f69e3349797fd74f99864d27716d6b48ac51734cb3bec03ea
previous_output_index=7
previous_output_amount=499550

## 2. Build & Sign transaction.

### A. Encode multisig output script.



In [23]:
echo $publickey_44h_1h_2h_0_0
echo $publickey_44h_1h_2h_0_1
echo $publickey_44h_1h_2h_0_2

# multisig_script=

multisig_script=$(bx script-encode "2 [03a151f572ab9462701f055904c7e80e12dd64cce5e59368e741139415a5e129cd] [0293798c3c194caf03cd12edb23351a94a69706638e712fadd3abaa0db791921a1] [02035765ecd6c51d151d3676bd2f87307030a0d5d3f32a4636707b345a444b074e] 3 checkmultisig")   
echo $multisig_script 


03a151f572ab9462701f055904c7e80e12dd64cce5e59368e741139415a5e129cd
0293798c3c194caf03cd12edb23351a94a69706638e712fadd3abaa0db791921a1
02035765ecd6c51d151d3676bd2f87307030a0d5d3f32a4636707b345a444b074e
522103a151f572ab9462701f055904c7e80e12dd64cce5e59368e741139415a5e129cd210293798c3c194caf03cd12edb23351a94a69706638e712fadd3abaa0db791921a12102035765ecd6c51d151d3676bd2f87307030a0d5d3f32a4636707b345a444b074e53ae


### B. Wrap multsig in P2SH output script.

**Derive p2sh address from multisig script.**

In [24]:
# Derive address to be use later to watch address.
bx script-to-address "2 [03a151f572ab9462701f055904c7e80e12dd64cce5e59368e741139415a5e129cd] [0293798c3c194caf03cd12edb23351a94a69706638e712fadd3abaa0db791921a1] [02035765ecd6c51d151d3676bd2f87307030a0d5d3f32a4636707b345a444b074e] 3 checkmultisig" \
| bx address-decode


wrapper
{
    checksum 3615012287
    payload a1d311cb452251c45fe6a220e6a60c5272b92d7b
    version 196
}


**Derive p2sh(multisig) script.**

In [25]:
# bx bitcoin160 [encoded script]

# embedded_script_hash=
embedded_script_hash=$(bx bitcoin160 $multisig_script)
echo $embedded_script_hash


a1d311cb452251c45fe6a220e6a60c5272b92d7b


In [26]:
# output_script=

output_script=$(bx script-encode "hash160 [a1d311cb452251c45fe6a220e6a60c5272b92d7b] equal")
echo $output_script


a914a1d311cb452251c45fe6a220e6a60c5272b92d7b87


### C. Compute output value.

In [27]:
# tx bytes: 4 + 1 + 1*(32+4+1+1+72+1+33+4) + 1 + 1*(8+1+1+1+20+1) + 4 

# tx_byte_count=

tx_byte_count=$(expr 4 + 1 + 1 \* 148 + 1 + 1 \* 32 + 4)
echo $tx_byte_count

190


In [28]:
# Output amount total.

# output_amount=

output_amount=$(expr $previous_output_amount - 0 \* 400 - $tx_byte_count \* 1)
echo $output_amount


499360


### D. Construct transaction.

In [29]:
# bx tx-encode \
# --input [previous tx id]:[index]:[sequence] \
# --output [output script]:[output amount]

# my_tx=

my_tx=$(bx tx-encode \
--input $previous_txid:$previous_output_index:4294967295 \
--output $output_script:$output_amount)


### E. Sign transaction & finalise transaction.

**Fetch previous output script.**

In [30]:
# previous_output_script=

previous_output_script=$(bx fetch-tx --format json $previous_txid \
| jq -r ".transaction.outputs[7].script")

echo $previous_output_script

dup hash160 [3054cf92dc80c6efd7559e660bde8b0fa7a85adc] equalverify checksig


**Sign transaction.**

In [31]:
# bx input-sign [private key] "previous output script" [transaction template]

# signature=

signature=$(bx input-sign --sign_type all --index 0 $privatekey_44h_1h_1h_0_7 "$previous_output_script" $my_tx)


**Set input into finalised transaction**

In [32]:
# bx input-set "[signature] [public key point]" [transaction template]

# my_tx=

my_tx=$(bx input-set --index 0 "[$signature] [$publickey_44h_1h_1h_0_7]" $my_tx)
echo $my_tx | bx tx-decode -f json | jq


{
  "transaction": {
    "hash": "c984d7c2cd330f8a980a8953f3b6141b6441933054838eb09c29f1709775da7c",
    "inputs": [
      {
        "address_hash": "3054cf92dc80c6efd7559e660bde8b0fa7a85adc",
        "previous_output": {
          "hash": "11a8587f5077fc7f69e3349797fd74f99864d27716d6b48ac51734cb3bec03ea",
          "index": "7"
        },
        "script": "[30440220404e4fab19409ff26ef31a6b75440063876a05b5508a7a7f53e7104548af33b2022075aecf64eaf73014b7b59998144323395ba6f4b42e8fcefbcb30ee13cd2e5e7701] [02eecb5f8cb9b6dfe418b96147976725cbc2f8ce6de4361ea1b8a995a7ef7765fc]",
        "sequence": "4294967295"
      }
    ],
    "lock_time": "0",
    "outputs": [
      {
        "address_hash": "a1d311cb452251c45fe6a220e6a60c5272b92d7b",
        "script": "hash160 [a1d311cb452251c45fe6a220e6a60c5272b92d7b] equal",
        "value": "499360"
      }
    ],
    "version": "1"
  }
}


In [33]:
# Validate.
bx validate-tx $my_tx

matching transaction with unspent outputs


: 255

In [17]:
# Send.
bx send-tx $my_tx

Sent transaction.
